## ML Ridge Model

In [1]:
# evaluate an ridge regression model on the dataset

import os 

import pandas as pd
import pandas as pd
from pandas import read_csv


import numpy as np
from numpy import arange
from numpy import mean
from numpy import std
from numpy import absolute

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
from collections import Counter

import sklearn
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)
#pd.set_option('display.max_columns', None) # spread dataframe columns 





In [221]:
def ridg_prediction(df, row2, split):
    
    # load the dataset
    data = df.values
    X, y = data[:, 1:], data[:, :1]

    # define model
    model = Ridge()
    
    # define model evaluation method
    cv = RepeatedKFold(n_splits=split, n_repeats=3, random_state=1)
    
    # define grid
    grid = dict()
    grid['alpha'] = arange(0, 1, 0.01)
    
    # define search
    search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    
    # perform the search
    results = search.fit(X, y)
    
    # summarize
    print('MAE: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)

    # make a prediction
    yhat = results.predict([row2])

    # summarize prediction
    print('Predicted: %.3f' % yhat)
    
    return yhat

In [222]:
#Return list of states in US
def states_list(df_kff):
    
    states = [] #list of states
    for state in df_kff.State:
        if state in states:
            continue
        else:
            states.append(state)
    return states

In [236]:
#The function gets crime and for each crime will Ridge for each state
#Out put: return a predicted list of spesific crime for each state
def preper_for_ridge(df,states, crime):

    #groupd state
    grouped = df.groupby(df.State)
    dic_predict = {}
    list_predict = []

    for state in states:
        curr = grouped.get_group(state)#state from 2010-2019
        split = len(curr['Year'])
        x=curr.copy()

        x.drop(['Year', 'State', crime,'Governor Political Affiliation'],axis=1, inplace=True)
        row2 = x.iloc[-1]

        curr=curr.drop(curr.index[[-1,-1]])
        curr.drop(['Year', 'State','Governor Political Affiliation'],axis=1, inplace=True)
        curr.iloc[:,1:2]

        # shift column to first position
        first_column = curr.pop(crime)
        curr.insert(0, crime, first_column)
        print("split=",split)
        print("state=",state)

        pediction = ridg_prediction(curr, row2, split-1)
        dic_predict[state]=pediction[0][0]
        list_predict.append(pediction[0][0])
        print("\n")
    return list_predict



/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [237]:


# Get data
path='/Users/barbenshabat/Downloads/KFF_handling/'
name='df_Y_S_KFF.xlsx'
df_kff=pd.read_excel(path+name, index_col=0) # read each file in the folder
df_kff.shape

# Copy the data
df_sklearn = df_kff.copy()

# Apply normalization techniques
cols=df_kff.drop({'Governor Political Affiliation'},axis=1).iloc[:,2:].columns #all relevant numeric

  
# Normlize
for column in cols: 
    df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1))

crime_list = df_kff.iloc[:,3:13].columns
states=[]
list_of_prediction_lists = []

#List of states in the US
for state in df_kff.State:
    if state in states:
        continue
    else:
        states.append(state)

for crime in crime_list:
    print("---------crime---------------------------------", crime)
    list_of_prediction_lists.append(preper_for_ridge(df_sklearn,states,crime))



/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/xlrd/xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():


---------crime--------------------------------- Violent crime
split= 8
state= ALABAMA


/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


MAE: -0.007
Config: {'alpha': 0.0}
Predicted: 0.097


split= 9
state= ALASKA
MAE: -0.003
Config: {'alpha': 0.03}
Predicted: 0.037


split= 10
state= ARIZONA


/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.09566e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True,


...

/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.64092e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True,


MAE: -0.009
Config: {'alpha': 0.0}
Predicted: 0.079


split= 10
state= WYOMING
MAE: -0.002
Config: {'alpha': 0.99}
Predicted: 0.006


split= 9
state= ILLINOIS


/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.4335e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True,


MAE: -0.012
Config: {'alpha': 0.0}
Predicted: 0.138


split= 9
state= OREGON
MAE: -0.024
Config: {'alpha': 0.01}
Predicted: 0.075


split= 7
state= MINNESOTA
MAE: -0.010
Config: {'alpha': 0.0}
Predicted: 0.059




/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.17769e-17): result may not be accurate.
  dual_coef = linalg.solve(K, y, sym_pos=True,


In [238]:
predictions_df = pd.DataFrame(states, columns =['State'])
df19 = df_sklearn[df_sklearn['Year']== 2019] #shape (48,100)

for idx,crime in enumerate(crime_list):
    pred = list_of_prediction_lists[idx]
    actu = df19.loc[:,crime]
    predictions_df.insert(len(predictions_df.columns),"Pred "+crime,pred) #shape (51,11)
    
    
x = list(set(states).difference(df19['State']))
y_48_11=predictions_df.copy()

predictions_df["Sum pred"] = predictions_df.sum(axis=1)

for val in x:
    y_48_11.drop(y_48_11.loc[y_48_11['State']==val].index, inplace=True)#shape (48,11)
    
y_48_11["Sum pred"] = y_48_11.sum(axis=1)

map_df = pd.DataFrame(y_48_11, columns =['State']) #shape (48,1)

for idx,crime in enumerate(crime_list):
    pred = y_48_11.iloc[:,idx+1]
    actu = list(df19.iloc[:,idx+2])
    map_df.insert(len(map_df.columns),"Pred "+crime,pred)
    map_df.insert(len(map_df.columns),"Actual "+crime,actu)    
    map_df.insert(len(map_df.columns),"deviation "+crime,abs(actu-pred))
    div = (map_df["deviation "+crime] / map_df["deviation "+crime].sum()) * 100
    map_df.insert(len(map_df.columns),"deviation "+crime+" %",div)
    
for i in range (3):
    entry = predictions_df.loc[predictions_df['State'] == x[i]]
    map_df = pd.concat([map_df,entry])
    
map_df["Sum pred"] = predictions_df["Sum pred"]
map_df=map_df.round(decimals=3)
map_df=map_df.sort_values(by=[map_df.columns[0]])
map_df.to_excel('Ridge model all crime.xlsx',engine='xlsxwriter')
map_df.head()

/Users/barbenshabat/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
divper = pd.DataFrame(map_df, columns = ['deviation Violent crime %',
 'deviation Murder and nonnegligent manslaughter %',
 'deviation Forcible rape %',
 'deviation Robbery %',
 'deviation Aggravated assault %',
 'deviation Property crime %',
 'deviation Burglary %',
 'deviation Larceny-theft %',
 'deviation Motor vehicle theft %',
 'deviation Arson1 %'])
divper.describe()